## Import the necessary library

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
## 
api_key = os.getenv('HF_TOKEN')

In [4]:
## Load the pdf file
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("D:\LLM\Project1\PDF\An_Efficient_Machine_Learning_Approach_Analysis_of_Supervised_Machine_Learning_M.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'source': 'D:\\LLM\\Project1\\PDF\\An_Efficient_Machine_Learning_Approach_Analysis_of_Supervised_Machine_Learning_M.pdf', 'page': 0}, page_content='An Efficient Machine Learning Approach: Analysis \nof Supervised Machine Learning Methods to Forecast \nthe Diamond Price\nMd Shaik Amzad Basha \nGITAM School of Business. \nGandhi Institute of Technology and \nManagement (Deemed to be University) \nBengaluru, India \namjuamjad66@gmail.com Peerzadah Mohammad Oveis  \nGITAM School of Business. \nGandhi Institute of Technology and \nManagement (Deemed to be University) \nBengaluru, India\npeerzadahmohammad@gmail.com C Prabavathi \nDepartment of Professional Studies \nChrist (Deemed to be University) \nBengaluru, India \nprabavathidps@gmail.com\nMacherla Bhagya Lakshmi \nDepartment of Professional Studies \nChrist (Deemed to be University)\nBengaluru, India \nbhagyamacherladps@gmail.com M Martha Sucharitha \nDepartment of Professional Studies \nChrist (Deemed to be Universi

## Split the documents 

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size =2000,chunk_overlap=100).split_documents(docs)
text_splitter

[Document(metadata={'source': 'D:\\LLM\\Project1\\PDF\\An_Efficient_Machine_Learning_Approach_Analysis_of_Supervised_Machine_Learning_M.pdf', 'page': 0}, page_content="An Efficient Machine Learning Approach: Analysis \nof Supervised Machine Learning Methods to Forecast \nthe Diamond Price\nMd Shaik Amzad Basha \nGITAM School of Business. \nGandhi Institute of Technology and \nManagement (Deemed to be University) \nBengaluru, India \namjuamjad66@gmail.com Peerzadah Mohammad Oveis  \nGITAM School of Business. \nGandhi Institute of Technology and \nManagement (Deemed to be University) \nBengaluru, India\npeerzadahmohammad@gmail.com C Prabavathi \nDepartment of Professional Studies \nChrist (Deemed to be University) \nBengaluru, India \nprabavathidps@gmail.com\nMacherla Bhagya Lakshmi \nDepartment of Professional Studies \nChrist (Deemed to be University)\nBengaluru, India \nbhagyamacherladps@gmail.com M Martha Sucharitha \nDepartment of Professional Studies \nChrist (Deemed to be Universi

## Creating the huggingface Embeddings

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\LLM\Project1\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\LLM\Project1\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Creating the vector store

In [8]:
from langchain_community.vectorstores import FAISS

In [9]:
store = FAISS.from_documents(text_splitter,embeddings)
retriver = store.as_retriever()

## Creating a prompt

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
"""
Based on the {context} provided answer the query asked by the user in a best possible way.
Example1- Question:"What skill is necessary to become Data Scientist?"
Answer:"SQL, Python, Machine Learning and concepts which help in future values predictions."
Question:{input}
Answer:
"""
)

## Intialize the LLM Model

In [13]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=api_key,model="llama3-8b-8192")

## Create a chain 

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

combine_docs_model_with_prompt = create_stuff_documents_chain(llm,prompt)
retrival_chain = create_retrieval_chain(retriver,combine_docs_model_with_prompt)

In [20]:
response = retrival_chain.invoke({'input':"give me the figure of diamond dimensions"})
print(response['answer'])

The article discusses the 4Cs of diamond quality, which are color, clarity, carat, and cut. These factors are used to evaluate the quality of a diamond and determine its price. The article also discusses the use of machine learning techniques to predict the price of a diamond based on these factors.

The authors of the article trained several machine learning models, including linear regression, random forest, decision tree, cat-boost regressor, and XGB regressor, using a dataset of 53,940 diamond entries. The dataset included information on the cut, color, clarity, carat, and dimensions of each diamond, as well as its price.

The authors found that the random forest model performed the best, with a mean absolute error (MAE) of $243.15. They also found that the carat and cut of the diamond were the most important factors in determining its price, followed by the color and clarity.

The article concludes that machine learning techniques can be used to accurately predict the price of a d